# Clean database

In [2]:
import os
import pandas as pd
from tqdm import tqdm

## move file

In [2]:
db_path = '/app/nfs_clientshare/Datasets/temp_db/2024-02-28-ส่งเทรนรูป-8'
new_db_path = '/app/nfs_clientshare/Datasets/temp_db'

In [19]:
def is_barcode(file):
    if set('arcodบาร์์โค้ด') & set(file.lower()):
        return True
    return False

In [20]:
def check_dir_exit(id, model):
    if not os.path.exists(f'{new_db_path}/{model}/{id}'):
        os.makedirs(f'{new_db_path}/{model}/{id}')

In [25]:
for root, dirs, files in tqdm(os.walk(db_path)):
    for file in files:
        product_id = os.path.basename(root)
        if is_barcode(file):
            check_dir_exit(product_id, 'barcode')
            if os.path.exists(f'{new_db_path}/barcode/{product_id}/{file}'):
                print(f'{root}/{file}')
            else:
                os.rename(f'{root}/{file}', f'{new_db_path}/barcode/{product_id}/{file}')
        else:
            check_dir_exit(product_id, 'product')
            if os.path.exists(f'{new_db_path}/product/{product_id}/{file}'):
                print(f'{root}/{file}')
            else:
                os.rename(f'{root}/{file}', f'{new_db_path}/product/{product_id}/{file}')

690it [00:12, 56.50it/s]


In [4]:
product_path = new_db_path + '/product'
ls_data = []
for root, dirs, files in tqdm(os.walk(product_path)):
    for file in files:
        id = os.path.basename(root)
        image_path = root+'/'+file
        ls_data.append([id, image_path])

0it [00:00, ?it/s]

50it [00:00, 161.59it/s]


In [5]:
ls_data = pd.DataFrame(ls_data, columns=['id', 'image_path'])
ls_data['id'] = ls_data['id'].astype(int)

In [3]:
from PIL import Image

In [30]:
for index, row in tqdm(ls_data.iterrows()):
    try:
        img = Image.open(row['image_path'])
        img.verify()
    except:
        print(row['image_path'])

9603it [00:26, 362.89it/s]


## check duplicate

In [6]:
current_code = pd.read_csv('result.csv')
new_barcode = pd.read_csv('barcode_found.csv')
old_barcode = pd.read_csv('barcode_old_found.csv')

In [21]:
duplicate_index = old_barcode[old_barcode['CODE'].isin(current_code['code'])].index


In [22]:
old_barcode.drop(duplicate_index, inplace=True)

In [27]:
duplicate_id_index = old_barcode[old_barcode['id'].isin(current_code['product_id'])].index

In [28]:
old_barcode.drop(duplicate_id_index, inplace=True)

In [32]:
old_barcode

,id,image_path,CODE
0,44286,/app/nfs_clientshare/Datasets/cosmenet_scanpro...,689304181891
1,44286,/app/nfs_clientshare/Datasets/cosmenet_scanpro...,689304181884
2,42798,/app/nfs_clientshare/Datasets/cosmenet_scanpro...,8859572899930
3,44243,/app/nfs_clientshare/Datasets/cosmenet_scanpro...,810763038949
4,36648,/app/nfs_clientshare/Datasets/cosmenet_scanpro...,4548863002022
...,...,...,...
246,44559,/app/nfs_clientshare/Datasets/cosmenet_scanpro...,815402021269
247,47397,/app/nfs_clientshare/Datasets/cosmenet_scanpro...,18084014363
248,39721,/app/nfs_clientshare/Datasets/cosmenet_scanpro...,5939141217
249,37794,/app/nfs_clientshare/Datasets/cosmenet_scanpro...,6917246013357


In [37]:
new_barcode.image_path[0]

'/app/nfs_clientshare/Datasets/temp_db/barcode/36679/36679_Barcode#3.7ml..jpg'

In [55]:
new_scan = os.listdir('/app/nfs_clientshare/Datasets/temp_db/product')

In [57]:
new_scan = pd.DataFrame(new_scan, columns=['id'])

In [60]:
list_file = []
for root, dirs, files in tqdm(os.walk('/app/nfs_clientshare/Datasets/temp_db/product')):
    for file in files:
        pid = os.path.basename(root)
        list_file.append([pid, root+'/'+file, file])
list_file = pd.DataFrame(list_file, columns=['id', 'image_path', 'file_name'])

683it [00:03, 184.05it/s]


In [59]:
link = 'https://chat.cosmenet.in.th/chatbot/upload/product/scan/'

In [62]:
list_file['full_path'] = link + list_file['id'].astype(str) + '/' + list_file['file_name']

In [68]:
import requests

In [73]:
def insert_data(body, tag_name):
    response = requests.post(
        url="http://10.148.0.15:4445/" + tag_name + "/insert", 
        json=body,
        headers={
            "Content-Type": "application/json", 
            'Authorization': 'Bearer dev'
            },
    )
    fail = response.json()['datas_failed']
    if len(fail) > 0:
        print(fail)
        return False
    return response

In [74]:
def find_link_id(id):
    response = requests.get(
            url=f"http://10.148.0.15:4445/product?page_start=1&page_size=10&filter_fields=%27_id%27&filter=%7B%22product_id%22%3A{id}%7D&all_data=false", 
            headers={
                "Content-Type": "application/json", 
                'Authorization': 'Bearer dev'
                },
    )
    return response.json()['datas'][0]['id']

In [78]:
scan_body = {
  "image_path": "",
  "product_link": "",
  "update_by": "music",
}

for i in tqdm(range(len(list_file))):
  prouct_id = str(list_file.iloc[i]['id'])
  scan_body['image_path'] = list_file.iloc[i]['full_path']
  scan_body['product_link'] = find_link_id(prouct_id)
  if not insert_data(scan_body, 'scan'):
      print(f'index {i}')
      break

100%|██████████| 9603/9603 [3:39:45<00:00,  1.37s/it]  


# Join data

In [46]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from IPython.display import display ,HTML
import requests, os
from PIL import Image
from io import BytesIO
from pyzbar.pyzbar import decode
from tqdm.auto import tqdm
from pathlib import Path

b_file_path = 'raw_data_production/b_file.csv'
b_iblock_element_path = 'raw_data_production/b_iblock_element.csv'
b_iblock_element_property_path = 'raw_data_production/b_iblock_element_property.csv'
b_iblock_property_path = 'raw_data_production/b_iblock_property.csv'
b_iblock_section_element_path = 'raw_data_production/b_iblock_section_element.csv'
b_iblock_section_path = 'raw_data_production/b_iblock_section.csv'
b_iblock_path = 'raw_data_production/b_iblock.csv'

b_iblock_element_property = pd.read_csv(b_iblock_element_property_path, usecols=['ID', 'IBLOCK_PROPERTY_ID', 'IBLOCK_ELEMENT_ID', 'VALUE'])
b_file = pd.read_csv(b_file_path, usecols=['ID', 'SUBDIR', 'FILE_NAME', 'ORIGINAL_NAME'])
b_iblock_element = pd.read_csv(b_iblock_element_path, usecols=['ID', 'IBLOCK_ID', 'IBLOCK_SECTION_ID', 'ACTIVE', 'NAME', 'PREVIEW_PICTURE', 'TIMESTAMP_X'])
b_iblock_property = pd.read_csv(b_iblock_property_path, usecols=['ID', 'IBLOCK_ID', 'NAME', 'ACTIVE'])
b_iblock_section_element = pd.read_csv(b_iblock_section_element_path, usecols=['IBLOCK_SECTION_ID', 'IBLOCK_ELEMENT_ID'])
b_iblock_section = pd.read_csv(b_iblock_section_path, usecols=['ID', 'IBLOCK_ID', 'IBLOCK_SECTION_ID', 'NAME', 'ACTIVE', 'DEPTH_LEVEL', 'TIMESTAMP_X'])
b_iblock = pd.read_csv(b_iblock_path, usecols=['ID', 'IBLOCK_TYPE_ID', 'NAME'])

url_404 = 'https://viterbischool.usc.edu/wp-content/uploads/2023/10/404.jpg'
oops_url = 'https://colorlib.com/wp/wp-content/uploads/sites/2/404-error-page-templates.jpg'
cosmenet_upload_url_prefix = 'https://www.cosmenet.in.th/upload/'
product_url_prefix = 'https://www.cosmenet.in.th/product/'

# Filter only brand and drop maker
brand_and_maker_iblock_index = b_iblock_section['IBLOCK_ID'] == 17
brand_depth_index = b_iblock_section['DEPTH_LEVEL'] == 2
brand_data = b_iblock_section[brand_and_maker_iblock_index & brand_depth_index]
brand_data = brand_data.drop(columns=['IBLOCK_ID', 'DEPTH_LEVEL'])
brand_data.rename(columns={'ID': 'BID', 'NAME': 'BRAND_NAME', 'IBLOCK_SECTION_ID':'MID'}, inplace=True)
brand_data = brand_data.astype({'MID': 'int32'})
# Check for duplicate brand name
brand_duplicate_unique = brand_data[brand_data['BRAND_NAME'].duplicated()].sort_values(by='BRAND_NAME')['BRAND_NAME'].unique()
brand_duplicate = brand_data[brand_data['BRAND_NAME'].isin(brand_duplicate_unique)]\
    .sort_values(['BRAND_NAME', 'BID'])[['BID', 'BRAND_NAME']]
# brand not active
brand_not_active_index = brand_data['ACTIVE'] == 'N'
brand_not_active_data = brand_data[brand_not_active_index]
brand_not_active_data = brand_not_active_data.drop(columns=['ACTIVE'])
# brand active
brand_active_index = brand_data['ACTIVE'] == 'Y'
brand_active_data = brand_data[brand_active_index]
brand_active_data = brand_active_data.drop(columns=['ACTIVE'])

brand_data.reset_index(drop=True, inplace=True)

# Filter only category and subcategory
cat_and_subcat_block_index = b_iblock_section['IBLOCK_ID'] == 6
cat_subcat_data = b_iblock_section[cat_and_subcat_block_index]
cat_subcat_data = cat_subcat_data.drop(columns=['IBLOCK_ID'])
# not active
cat_subcat_not_active_index = cat_subcat_data['ACTIVE'] == 'N'
cat_subcat_not_active_data = cat_subcat_data[cat_subcat_not_active_index]
cat_subcat_not_active_data = cat_subcat_not_active_data.drop(columns=['ACTIVE'])
# active
cat_subcat_active_index = cat_subcat_data['ACTIVE'] == 'Y'
cat_subcat_active_data = cat_subcat_data[cat_subcat_active_index]
cat_subcat_active_data = cat_subcat_active_data.drop(columns=['ACTIVE'])

# Filter only category and drop subcategory
category_data = cat_subcat_data[cat_subcat_data['DEPTH_LEVEL'] == 1]
category_data = category_data.drop(columns=['DEPTH_LEVEL', 'IBLOCK_SECTION_ID'])
category_data.rename(columns={'ID': 'CID', 'NAME': 'CATEGORY_NAME'}, inplace=True)
category_data.reset_index(drop=True, inplace=True)

# Merge category and subcategory
subcategory_data = pd.merge(cat_subcat_data, cat_subcat_data, left_on='IBLOCK_SECTION_ID', right_on='ID')
midcategory_index = subcategory_data['DEPTH_LEVEL_x'] == 2
subcategory_data = subcategory_data[~midcategory_index]
subcategory_data.drop(columns=['DEPTH_LEVEL_x', 'DEPTH_LEVEL_y', 'ID_y', 'NAME_y', 'IBLOCK_SECTION_ID_x', 'TIMESTAMP_X_y', 'ACTIVE_y'], inplace=True)
subcategory_data.rename(columns={'ID_x': 'SCID', 'NAME_x': 'SUBCATEGORY_NAME', 'IBLOCK_SECTION_ID_y': 'CID', 
                                 'TIMESTAMP_X_x':'TIMESTAMP_X', 'ACTIVE_x':'ACTIVE'}, inplace=True)
subcategory_data = subcategory_data.astype({'CID': 'int32'})
subcategory_data.reset_index(drop=True, inplace=True)

# Filter only product
product_block_index = b_iblock_element['IBLOCK_ID'] == 17
product_data = b_iblock_element[product_block_index]
product_data = product_data.drop(columns=['IBLOCK_ID'])
product_data.rename(columns={'ID': 'EID', 'IBLOCK_SECTION_ID': 'BID', 'NAME': 'PRODUCT_NAME'}, inplace=True)
# Check brand not provided
product_not_BID = product_data[product_data['BID'].isna()]
product_data = product_data.astype({'BID': 'int32'})
# create product url
product_data['PRODUCT_URL'] = product_data['EID'].apply(lambda x: product_url_prefix + str(x))
# merge product with category and subcategory
subcat_link_data = pd.merge(b_iblock_section_element, subcategory_data.drop(columns=['ACTIVE']), how='right', left_on='IBLOCK_SECTION_ID', right_on='SCID')
subcat_link_data.drop(columns=['IBLOCK_SECTION_ID', 'SUBCATEGORY_NAME', 'CID', 'TIMESTAMP_X'], inplace=True)
product_data_linked= pd.merge(product_data, subcat_link_data, how='left', left_on='EID', right_on='IBLOCK_ELEMENT_ID')
product_data_linked.drop(columns=['IBLOCK_ELEMENT_ID'], inplace=True)
# create product preview image
product_data_linked = pd.merge(product_data_linked, b_file, left_on='PREVIEW_PICTURE', right_on='ID', how='left')

product_data_linked['PREVIEW_PICTURE'] = product_data_linked.apply(lambda x: cosmenet_upload_url_prefix + x['SUBDIR'] 
                                                                   if not pd.isna(x['PREVIEW_PICTURE']) 
                                                                   else None, axis=1)
def check_url(x):
    if not pd.isna(x['FILE_NAME']):
        return x['PREVIEW_PICTURE'] + '/' + x['FILE_NAME']
    else:
        return None
product_data_linked['PREVIEW_PICTURE'] = product_data_linked.apply(lambda x: check_url(x), axis=1)
product_data_linked.drop(columns=['ID', 'SUBDIR', 'ORIGINAL_NAME'], inplace=True)
# active
product_active_index = product_data_linked['ACTIVE'] == 'Y'
product_active_data = product_data_linked[product_active_index]
product_active_data = product_active_data.drop(columns=['ACTIVE'])
product_active_data = product_active_data.astype({'SCID': 'int32'})
# not active
product_not_active_index = product_data_linked['ACTIVE'] == 'N'
product_not_active_data = product_data_linked[product_not_active_index]
product_not_active_data = product_not_active_data.drop(columns=['ACTIVE'])

product_data_linked.reset_index(drop=True, inplace=True)
product_data_linked.head(1)

,EID,TIMESTAMP_X,BID,ACTIVE,PRODUCT_NAME,PREVIEW_PICTURE,PRODUCT_URL,SCID,FILE_NAME
0,102,2022-06-28 20:09:51,1497,Y,Beneficial Make Off Perfect Eye & Lip Makeup R...,https://www.cosmenet.in.th/upload/iblock/b09/B...,https://www.cosmenet.in.th/product/102,73.0,Beneficial-Make-Off-Perfect-Eye-Lip-Makeup-Rem...


In [ ]:
ls_data[~(ls_data['id'].isin(product_data_linked['EID']))]